<h1><center>
Data Collecting and Munging
</center></h1>
<h3><center>
Marcel Colvin
</center></h3>

In [18]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import math
from selenium import webdriver
from pandas.io.html import read_html
import re
import sqlalchemy as sqla
import sqlite3 as sql
requests_cache.install_cache('coll_cache')
%matplotlib inline

In [19]:
sqlite_file = 'FootballData.sqlite'
football_conn = sqla.create_engine('sqlite:///' + sqlite_file)

## Get 1st Division English Soccer winners through time:
#### Data Source: https://en.wikipedia.org/wiki/List_of_English_football_champions

In [20]:
pl_standings_page = 'https://en.wikipedia.org/wiki/List_of_English_football_champions'
pl_standings_table = read_html(pl_standings_page, index_col=0, attrs={"class":"wikitable"})

### When it was called the First Division:

In [21]:
first_div_winners = pl_standings_table[1]
first_div_winners = first_div_winners.drop(first_div_winners.index[24])
first_div_winners = first_div_winners.drop(first_div_winners.index[44])

first_div_winners = first_div_winners.reset_index()
first_div_winners.columns = first_div_winners.iloc[0]
first_div_winners = first_div_winners[1:]

first_div_winners["Champions (number of titles)"] = [re.sub("[^A-Za-z]", " ", s).strip() for s in first_div_winners["Champions (number of titles)"]]
first_div_winners = first_div_winners.reset_index(drop= True)
first_div_winners.to_sql('first_div_winners',football_conn,if_exists='replace', index= False)

### For the Premier League:

In [6]:
pl_winners = pl_standings_table[2]
pl_winners = pl_winners.reset_index()
pl_winners.columns = pl_winners.iloc[0]
pl_winners = pl_winners[1:]
pl_winners["Champions (number of titles)"] = [re.sub("[^A-Za-z]", " ", s).strip() for s in pl_winners["Champions (number of titles)"]]
pl_winners = pl_winners.reset_index(drop = True)
pl_winners.to_sql('pl_winners',football_conn,if_exists='replace', index= False)

### Combining Both Together:

In [7]:
full_winners = pd.concat([first_div_winners[["Year","Champions (number of titles)", "Runners-up", "Third place"]], pl_winners[["Year","Champions (number of titles)", "Runners-up", "Third place"]]])
full_winners = full_winners.reset_index(drop =True)
full_winners.to_sql('full_EPL_winners',football_conn,if_exists='replace', index= False)

### Total Number of Titles per Team:

In [8]:
titles = pl_standings_table[3]
titles = titles.reset_index()
titles.columns = titles.iloc[0]
titles = titles[1:]
titles = titles.reset_index(drop = True)
for i in range(len(titles)):
    titles["Winners"][i] = int(titles["Winners"][i])
    titles["Runners-up"][i] = int(titles["Runners-up"][i])
titles.to_sql('titles',football_conn,if_exists='replace', index= False)

## Get Results for the 2015-2016 Season:
#### Data Source: https://en.wikipedia.org/wiki/2015%E2%80%9316_Premier_League

In [9]:
results_page = 'https://en.wikipedia.org/wiki/2015%E2%80%9316_Premier_League'
season_results_table = read_html(results_page, index_col=0, attrs={"class":"wikitable"})

### Final Table for the season:

In [14]:
season_results = season_results_table[3]
season_results = season_results.reset_index()
season_results.columns = season_results.iloc[0]
season_results = season_results[1:]
season_results = season_results.reset_index(drop = True)
season_results.to_sql('EPLseason_results_2015',football_conn,if_exists='replace', index= True)

### Matches in a Home/Away Matrix

In [16]:
matches = season_results_table[4]
matches.columns = matches.iloc[0]
matches = matches[1:]
matches.columns.names = ['Away']
matches.index.names = ['Home']
matches.to_sql('EPLmatches_2015',football_conn, index = True,if_exists='replace')

## Read in Leicester City's Standings over Time:
#### Data Source: https://en.wikipedia.org/wiki/List_of_Leicester_City_F.C._seasons

In [15]:
lc_standings = pd.read_csv("raw_data/lc_rank.csv")
lc_standings.to_sql('lc_rank',football_conn,if_exists='replace', index= False)

## Get Premier Leauge Statistics by team 2000-2016
#### Data Source: https://www.premierleague.com/stats/top/clubs/wins?se=-1

In [ ]:
stats_needed =[ 'losses', 'goals', 'total_yel_card', 'total_red_card','total_pass','total_scoring_att','total_tackle','saves', 'big_chance_missed','clean_sheet', 'total_offside', 'dispossessed', 'touches', 'att_ibox_goal','att_obox_goal','goals_conceded' ]
years_needed = ['9','10','11', '12', '13', '14', '15', '16','17','18','19','20','21','22', '27','42' ]

In [ ]:
#Creates a cache
options = webdriver.ChromeOptions()
options.add_argument("user-data-dir = C:\\Users\\marce\\AppData\\Local\\Temp\\scoped_dir3244_20158")

In [ ]:
def pull_yearly_stats(year):
    """This function takes in a specific year value that is found in the se parameter
    in the url of premierleague.com/stats/top/clubs and returns a dataframe
    of statistics that can be specified in the stats_needed value above"""
    browser.get("https://www.premierleague.com/stats/top/clubs/{}?se={}".format('wins', year))
    time.sleep(5)
    html = browser.page_source
    soup = BeautifulSoup(html,'lxml-xml')
    table = soup.find_all(name='tbody',attrs={"class": "statsTableContainer"})
    winstable = table[0]
    stat1 = winstable.find_all('td', attrs = {'class':'mainStat'})
    name1 = winstable.find_all('a', attrs = {'class': 'playerName'})
    t1 = [x.text.strip() for x in name1]
    t2 = [x.text.strip() for x in stat1]
    frame1 = pd.DataFrame({'Name': t1, 'wins':t2})
    for stat in stats_needed:
        browser.get("https://www.premierleague.com/stats/top/clubs/{}?se={}".format(stat, year))
        time.sleep(5)
        html = browser.page_source
        soup = BeautifulSoup(html,'lxml-xml')
        table = soup.find_all(name='tbody',attrs={"class": "statsTableContainer"})
        winstable = table[0]
        stat1 = winstable.find_all('td', attrs = {'class':'mainStat'})
        name1 = winstable.find_all('a', attrs = {'class': 'playerName'})
        t1 = [x.text.strip() for x in name1]
        t2 = [x.text.strip() for x in stat1]
        frame1 = frame1.merge(pd.DataFrame.from_dict({'Name':t1, stat:t2}), how = 'left', on = ["Name"])
    return frame1

In [ ]:
browser = webdriver.Chrome(executable_path ='C:\\Users\\marce\\Documents\\PythonProjects\\chromedriver_win32\\chromedriver.exe', chrome_options = options)
season_stats = [pull_yearly_stats(x) for x in years_needed ]

### Add the wages of each club to the statistics dataframe for each year:
#### Data Sources: 
http://www.telegraph.co.uk/sport/football/competitions/premier-league/9514149/Graphic-Premier-League-transfer-and-wage-spending-from-the-200001-season-to-201112.html

http://www.theguardian.com/football/2013/apr/18/premier-league-finances-club-by-club

https://www.theguardian.com/football/2014/may/01/premier-league-accounts-club-by-club-david-conn

https://www.theguardian.com/football/2015/apr/29/premier-league-finances-club-by-club

https://www.theguardian.com/football/2016/may/25/premier-league-finances-club-by-club-breakdown-david-conn

https://www.theguardian.com/football/2017/jun/01/premier-league-finances-club-by-club

In [ ]:
wages_raw = pd.read_csv("wages.csv")
wages= [0]*16
for i in range(0,16):
    wages[i] =wages_raw.loc[20*i:20*i+19]
    wages[i]["Name"] = wages[i]["Name"].str.lower()
wages[1]

In [ ]:
for i in range(0,16):
    season_stats[i]["Name"] = season_stats[i]["Name"].str.lower()
    season_stats[i] = season_stats[i].merge(wages[i], how = 'left', on = ["Name"] )

### Correct the Data

In [ ]:
for i in range(0,16):
    for j in range(0,20):
        if(math.isnan(season_stats[i]["wins"][j])):
            season_stats[i]["wins"][j] = 0
        if(math.isnan(season_stats[i]["losses"][j])):
            season_stats[i]["losses"][j] = 0
        if(math.isnan(season_stats[i]["total_red_card"][j])):
            season_stats[i]["total_red_card"][j] = 0
        if(math.isnan(season_stats[i]["clean_sheet"][j])):
            season_stats[i]["clean_sheet"][j] = 0
        if(season_stats[i]["Season Wage Bill (£mil)"][j]) == 0:
            season_stats[i]["Season Wage Bill (£mil)"][j] = np.nan

In [317]:
season_stats = [x.drop("saves", 1) for x in season_stats]#saves is too weird of a variable


In [24]:
for i in range(0,16):
    season_stats[i]["draws"] = (38 - season_stats[i]["wins"]- season_stats[i]["losses"])
    season_stats[i]["points"] = season_stats[i]["wins"] * 3 + (season_stats[i]["draws"])
    season_stats[i] = season_stats[i].sort_values(by = ["points"], ascending = [False])
    season_stats[i]["Won"] = False
    season_stats[i]["Won"].at[0] = True
    season_stats[i]["Top4"] = False
    season_stats[i]["Top4"].iloc[[0,1,2,3]] = True
    season_stats[i]["rel"] = False
    season_stats[i]["rel"].iloc[[17,18,19]] = True
    season_stats[i]["points_per_mil"] = season_stats[i]["points"] /season_stats[i]["Season Wage Bill (£mil)"]
    season_stats[i]["goal_diff"] = season_stats[i]["goals"] - season_stats[i]["goals_conceded"]
    if(i < 10):
        season_stats[i]["Year"] = int("200{}".format(i))
    else:
        season_stats[i]["Year"] = int("20{}".format(i))
    season_stats[i]["wage_normalized"] = (season_stats[i]["Season Wage Bill (£mil)"] - np.nanmean(season_stats[i]["Season Wage Bill (£mil)"]))/np.sqrt(np.nanvar(season_stats[i]["Season Wage Bill (£mil)"]))
    for j in range(len(season_stats[i])):
        if(season_stats[i]["total_pass"][j] != np.nan and season_stats[i]["total_pass"][j] != None):
            season_stats[i]["total_pass"][j] = float(str(season_stats[i]["total_pass"][j]).replace(',', ''))
        if(season_stats[i]["touches"][j] != np.nan and season_stats[i]["touches"][j] != None):
            season_stats[i]["touches"][j] = float(str(season_stats[i]["touches"][j]).replace(',', ''))
        if(season_stats[i]["total_tackle"][j] != np.nan and season_stats[i]["total_tackle"][j] != None):
            season_stats[i]["total_tackle"][j] = float(str(season_stats[i]["total_tackle"][j]).replace(',', ''))
    season_stats[i] = season_stats[i].reset_index(drop = True)


C:\Users\marce\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\marce\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\marce\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\marce\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value i

### Write to Database for later use:

In [25]:
for i in range(0,len(season_stats)): #final table for all seasons exported so it is easy to recompile
    season_stats[i].to_sql('pl_season_stats_{}'.format(i+1),football_conn,if_exists='replace', index = False)

## Get Player Statistics by Team for 2015-2016
#### Data Source = https://www.foxsports.com/soccer/stats?competition=1&season=20150&category=STANDARD

In [27]:
teams = ['arsenal', 'aston-villa', 'bournemouth', 'chelsea' ,'crystal-palace', 'everton','leicester-city', 'liverpool', 'manchester-city', 'manchester-united', 'newcastle-united', 'norwich-city',
         'southampton', 'stoke-city', 'sunderland', 'swansea-city', 'tottenham-hotspur', 'watford', 'west-bromwich-albion', 'west-ham-united']

In [28]:
season_stats_2015 = {}
for i in range(0,20):
    base_url = 'https://www.foxsports.com/soccer/{}-team-stats'.format(teams[i])
    params1 = {'competition':'1', 'season':'20150', 'category': 'STANDARD'}
    req = requests.get(base_url, params = params1)
    soupe = BeautifulSoup(req.text,'html.parser')
    cols = soupe.find_all('th')
    rows = soupe.find_all('tr', attrs = {'class': 'wisbb_fvStand'})
    t2 = [x.text.strip() for x in cols]
    a = pd.DataFrame(t2)
    for row in rows[1:]:
        values = row.find_all('td')
        t1 = [x.text.strip() for x in values]
        t1[0] = t1[0].split('\n')[2]
        a = pd.concat([a, pd.DataFrame(t1)], axis = 1)
    a = a.T
    a.columns = a.iloc[0]
    a = a.reset_index(drop = True)
    season_stats_2015[teams[i]] = a[1:]

In [31]:
for team in teams:
    season_stats_2015[team]["GP"] = [int(s) for s in season_stats_2015[team]["GP"]]
    season_stats_2015[team]["GS"] = [int(s) for s in season_stats_2015[team]["GS"]]
    season_stats_2015[team]["MP"] = [int(s) for s in season_stats_2015[team]["MP"]]
    season_stats_2015[team]["G"] = [int(s) for s in season_stats_2015[team]["G"]]
    season_stats_2015[team]["A"] = [int(s) for s in season_stats_2015[team]["A"]]
    season_stats_2015[team]["SOG"] = [int(s) for s in season_stats_2015[team]["SOG"]]
    season_stats_2015[team]["S"] = [int(s) for s in season_stats_2015[team]["S"]]

C:\Users\marce\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\marce\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\marce\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [32]:
for team in teams: #export the data for the 2015 season
    season_stats_2015[team].to_sql('{}_2015_stats'.format(team.replace('-','_')),football_conn,if_exists='replace', index= False)

# Get NBA Champions Data

In [9]:
NBA_url = 'https://en.wikipedia.org/wiki/List_of_NBA_champions'
NBA_website_tables = read_html(NBA_url, index_col=0, attrs={"class":"wikitable"})

In [31]:
NBA_winners = NBA_website_tables[0]
NBA_winners = NBA_winners.reset_index(drop = False)
NBA_winners.columns = NBA_winners.iloc[0]
NBA_winners = NBA_winners[1:]
NBA_winners = NBA_winners.reset_index(drop = True)
NBA_winners =NBA_winners[["Year", "Winning team", "Losing team"]]
NBA_winners["Winning team"]= [re.sub("[^A-Za-z]", " ", s).strip() for s in NBA_winners["Winning team"]]
NBA_winners["Losing team"]= [re.sub("[^A-Za-z]", " ", s).strip() for s in NBA_winners["Losing team"]]


NBA_winners.to_sql("NBA_winners", football_conn,if_exists='replace', index= False)

# Get MLS Champions Data

In [32]:
MLS_url = 'https://en.wikipedia.org/wiki/List_of_American_and_Canadian_soccer_champions'
MLS_website_tables = read_html(MLS_url, index_col=0, attrs={"class":"wikitable"})

In [44]:
MLS_winners = MLS_website_tables[3]
MLS_winners = MLS_winners.reset_index(drop = False)
MLS_winners.columns = MLS_winners.iloc[0]
MLS_winners = MLS_winners[1:]
MLS_winners = MLS_winners.reset_index(drop = True)
MLS_winners = MLS_winners[["Year", "MLS Cup (number of titles)", "Runners-up"]]
MLS_winners.columns = ["Year", "Champion", "Runners-up", "toRemove"]
MLS_winners = MLS_winners[["Year", "Champion", "Runners-up"]]
MLS_winners["Champion"]= [re.sub("[^A-Za-z]", " ", s).strip() for s in MLS_winners["Champion"]]

MLS_winners.to_sql("MLS_winners", football_conn,if_exists='replace', index= False)

# Get NFL Champions Data

In [46]:
NFL_url = 'https://en.wikipedia.org/wiki/List_of_Super_Bowl_champions'
NFL_website_tables = read_html(NFL_url, index_col=0, attrs={"class":"wikitable"})

In [117]:
NFL_winners = NFL_website_tables[1]
NFL_winners = NFL_winners.reset_index(drop = False)
NFL_winners.columns =NFL_winners.iloc[0]
NFL_winners = NFL_winners[1:]
NFL_winners = NFL_winners.reset_index(drop = True)
NFL_winners = NFL_winners[:-5]
NFL_winners= NFL_winners[["Date", "Winning team", "Losing team"]]
NFL_winners["Winning team"] = [s.split("0")[0].strip() for s in NFL_winners["Winning team"]]
NFL_winners["Losing team"] = [s.split("0")[0].strip() for s in NFL_winners["Losing team"]]
NFL_winners.at[51, "Losing team"] = "New England Patriots"
NFL_winners["Date"] = [int(str(s).split(',')[1]) for s in NFL_winners["Date"]]
NFL_winners.to_sql("NFL_winners", football_conn,if_exists='replace', index= False)

# Get MLB Champions Data

In [90]:
MLB_url = 'https://en.wikipedia.org/wiki/List_of_World_Series_champions'
MLB_website_tables = read_html(MLB_url, index_col=0, attrs={"class":"wikitable"})

In [116]:
MLB_winners = MLB_website_tables[0]
MLB_winners = MLB_winners.reset_index(drop = False)
MLB_winners.columns =MLB_winners.iloc[0]
MLB_winners = MLB_winners[1:]
MLB_winners = MLB_winners.reset_index(drop = True)
MLB_winners = MLB_winners[["Year", "Winning team", "Losing team"]]
MLB_winners["Winning team"] = [str(s).split("(")[0].strip() for s in MLB_winners["Winning team"]]
MLB_winners["Losing team"] = [str(s).split("(")[0].strip() for s in MLB_winners["Losing team"]]
MLB_winners["Winning team"] = [str(s).split("[")[0].strip() for s in MLB_winners["Winning team"]]
MLB_winners["Losing team"] = [str(s).split("[")[0].strip() for s in MLB_winners["Losing team"]]
MLB_winners = MLB_winners.drop(MLB_winners.index[[1,91]])
MLB_winners = MLB_winners.reset_index(drop = True)
MLB_winners.to_sql("MLB_winners", football_conn, if_exists = 'replace', index = False)

In [23]:
db.close()

#### Data sources:
https://en.wikipedia.org/wiki/List_of_Leicester_City_F.C._seasons
https://en.wikipedia.org/wiki/2015%E2%80%9316_Premier_League
https://en.wikipedia.org/wiki/List_of_English_football_champions
https://www.premierleague.com/stats/top/clubs/wins?se=-1
https://www.foxsports.com/soccer/stats?competition=1&season=20150&category=STANDARD
http://www.telegraph.co.uk/sport/football/competitions/premier-league/9514149/Graphic-Premier-League-transfer-and-wage-spending-from-the-200001-season-to-201112.html
http://www.theguardian.com/football/2013/apr/18/premier-league-finances-club-by-club
https://www.theguardian.com/football/2014/may/01/premier-league-accounts-club-by-club-david-conn
https://www.theguardian.com/football/2015/apr/29/premier-league-finances-club-by-club
https://www.theguardian.com/football/2016/may/25/premier-league-finances-club-by-club-breakdown-david-conn
https://www.theguardian.com/football/2017/jun/01/premier-league-finances-club-by-club
https://en.wikipedia.org/wiki/List_of_World_Series_champions
https://en.wikipedia.org/wiki/List_of_NBA_champions
https://en.wikipedia.org/wiki/List_of_American_and_Canadian_soccer_champions
https://en.wikipedia.org/wiki/List_of_Super_Bowl_champions